!!! Building a GAN-Based AI Text Detector !!!

PREPARATION DES DONNEES

In [ ]:
from google.colab import files
import pandas as pd

# Téléchargement manuel des fichiers CSV
uploaded = files.upload()

# Charger les fichiers dans des DataFrames
train_essays = pd.read_csv("train_essays.csv")
test_essays = pd.read_csv("test_essays.csv")
train_prompts = pd.read_csv("train_prompts.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Afficher un aperçu des données
print("Train Essays:")
print(train_essays.head())

print("\nTest Essays:")
print(test_essays.head())

print("\nTrain Prompts:")
print(train_prompts.head())

print("\nSample Submission:")
print(sample_submission.head())


Saving sample_submission.csv to sample_submission.csv
Saving test_essays.csv to test_essays.csv
Saving train_essays.csv to train_essays.csv
Saving train_prompts.csv to train_prompts.csv
Train Essays:
         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  

Test Essays:
         id  prompt_id          text
0  0000aaaa          2  Aaa bbb ccc.
1  1111bbbb          3  Bbb ccc ddd.
2  2222cccc          4  CCC ddd eee.

Train Prompts:
   prompt_id                       prompt_name  \
0          0       

 Étape 2 : Installer et charger PyTorch & Transformers

In [ ]:
# Installer PyTorch et Transformers si nécessaire
!pip install transformers torch torchvision torchaudio --quiet

# Importer les bibliothèques nécessaires
import torch
from transformers import BertTokenizer

# Détecter le GPU (si disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00
Utilisation de : cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Étape 3 : Prétraitement et tokenisation des textes

In [ ]:
# Fonction pour tokeniser les textes
def tokenize_texts(texts, tokenizer, max_length=512):
    return tokenizer(
        texts.tolist(),  # Transformer la série pandas en liste
        padding=True,  # Ajoute du padding pour uniformiser la taille des séquences
        truncation=True,  # Coupe les séquences trop longues (max_length)
        max_length=max_length,  # Fixe une longueur maximale des séquences
        return_tensors="pt"  # Retourne les tenseurs PyTorch (important pour l'entraînement)
    )

# Appliquer la tokenisation aux textes du train et test set
train_tokens = tokenize_texts(train_essays["text"], tokenizer)
test_tokens = tokenize_texts(test_essays["text"], tokenizer)

# Afficher les clés des tokens générés
print("Train Tokens Keys:", train_tokens.keys())


Train Tokens Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


. Résultat de la tokenisation

In [ ]:
print(train_tokens.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


Étape 4 : Préparer les données pour l'entraînement

1. Créer un Dataset PyTorch

In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

class GANDAIGDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# On split le dataset
texts = src_train["text"].tolist()
labels = src_train["label"].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=1-train_ratio)

train_dataset = GANDAIGDataset(train_texts, train_labels)
val_dataset = GANDAIGDataset(val_texts, val_labels)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(val_dataset, batch_size=test_batch_size)





5. Définition du Generator

In [ ]:
# Le Generator va prendre un bruit latent (vecteur aléatoire) et produire une représentation "fausse" de texte dans l’espace des embeddings BERT.

import torch.nn as nn
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertEncoder

config = BertConfig(num_hidden_layers=num_hidden_layers)

class Generator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, 256 * 128)  # pour transformer le bruit en matrice
        self.conv_net = nn.Sequential(
            nn.ConvTranspose1d(256, 128, kernel_size=3),
            nn.ReLU(),
            nn.ConvTranspose1d(128, 64, kernel_size=3),
            nn.ReLU(),
            nn.ConvTranspose1d(64, 32, kernel_size=3),
            nn.ReLU()
        )
        self.bert_encoder = BertEncoder(config)

    def forward(self, x):
        x = self.fc(x)  # [batch, 256*128]
        x = x.view(-1, 256, 128)  # reshape en [batch, channels, seq_len]
        x = self.conv_net(x)  # [batch, channels, seq_len]
        x = x.permute(0, 2, 1)  # [batch, seq_len, dim] pour BertEncoder
        attention_mask = torch.ones(x.shape[:2]).to(x.device)  # on suppose tout est pertinent
        encoded_output = self.bert_encoder(inputs_embeds=x, attention_mask=attention_mask)
        return encoded_output


6. Définition du Discriminator

In [ ]:
# Le Discriminator prend en entrée des embeddings (réels ou générés) et prédit si c’est du vrai texte ou généré.

class SumBertPooler(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        sum_hidden = hidden_states.sum(dim=1)
        sum_mask = sum_hidden.sum(1).unsqueeze(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_hidden / sum_mask
        return mean_embeddings

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_encoder = BertEncoder(config)
        self.bert_encoder.layer = nn.ModuleList([
            layer for layer in pretrained_model.bert.encoder.layer[:6]
        ])
        self.pooler = SumBertPooler()
        self.classifier = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1)
        )

    def forward(self, input):
        out = self.bert_encoder(inputs_embeds=input).last_hidden_state
        out = self.pooler(out)
        out = self.classifier(out)
        return torch.sigmoid(out).view(-1)


7. Entraînement GAN – avec AUC Tracking

Fonction preparation_embedding

Elle génère les embeddings BERT pour un batch de textes (réels).

In [ ]:
def preparation_embedding(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
    input_ids = encodings['input_ids'].to(device)
    token_type_ids = encodings['token_type_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    with torch.no_grad():
        embedded = embedding_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).last_hidden_state
    return embedded


Fonction d’évaluation AUC

In [ ]:
from sklearn.metrics import roc_auc_score

def eval_auc(model):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in test_loader:
            texts, labels = batch
            embeddings = preparation_embedding(texts)
            outputs = model(embeddings)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.numpy())

    auc = roc_auc_score(actuals, predictions)
    print("AUC:", round(auc, 4))
    return auc


✅ Initialisation des modèles et de l'entraînement

In [ ]:
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


✅ Fonction GAN_step

In [ ]:
def GAN_step(optimizerG, optimizerD, netG, netD, real_data, label, epoch, i):
    netD.train()
    netG.train()

    # === Train Discriminator ===
    netD.zero_grad()
    batch_size = real_data.size(0)

    output_real = netD(real_data)
    errD_real = criterion(output_real, label)
    errD_real.backward()
    D_x = output_real.mean().item()

    noise = torch.randn(batch_size, nz).to(device)
    fake_data = netG(noise).last_hidden_state.detach()  # detach to avoid gradients in G
    label_fake = torch.zeros_like(label)
    output_fake = netD(fake_data)
    errD_fake = criterion(output_fake, label_fake)
    errD_fake.backward()
    D_G_z1 = output_fake.mean().item()

    optimizerD.step()

    # === Train Generator ===
    netG.zero_grad()
    label_gen = torch.ones_like(label)
    fake_data = netG(noise).last_hidden_state
    output = netD(fake_data)
    errG = criterion(output, label_gen)
    errG.backward()
    D_G_z2 = output.mean().item()

    optimizerG.step()

    if i % 50 == 0:
        print(f"[{epoch}/{num_epochs}][{i}/{len(train_loader)}] Loss_D: {errD_real + errD_fake:.4f} Loss_G: {errG:.4f} D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}")

    return optimizerG, optimizerD, netG, netD


✅ Entraînement principal

In [ ]:
model_infos = []
for epoch in range(num_epochs):
    for i, batch in enumerate(train_loader):
        texts, labels = batch
        real_data = preparation_embedding(texts)
        labels = torch.tensor(labels).float().to(device)

        optimizerG, optimizerD, netG, netD = GAN_step(
            optimizerG=optimizerG,
            optimizerD=optimizerD,
            netG=netG,
            netD=netD,
            real_data=real_data,
            label=labels,
            epoch=epoch,
            i=i
        )

    auc_score = eval_auc(netD)
    model_infos.append({
        'epoch': epoch,
        'model_state_dict': netD.state_dict(),
        'auc_score': auc_score
    })

print("✅ Entraînement terminé !")


7. Inference & Soumission finale

In [ ]:
A. Sélectionner le meilleur modèle (selon AUC)

In [ ]:
# On récupère le modèle avec le meilleur score AUC
best_model_info = max(model_infos, key=lambda x: x['auc_score'])

# Charger le modèle
final_model = Discriminator().to(device)
final_model.load_state_dict(best_model_info['model_state_dict'])
final_model.eval()


B. Préparer les données de test

In [ ]:
# Prépare un Dataset pour la prédiction
class InferenceDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Dataset et DataLoader
sub_dataset = InferenceDataset(src_sub["text"].tolist())
inference_loader = DataLoader(sub_dataset, batch_size=test_batch_size)


C. Générer les prédictions

In [ ]:
sub_predictions = []

with torch.no_grad():
    for batch in inference_loader:
        embeddings = preparation_embedding(batch)
        embeddings = embeddings.to(device)

        outputs = final_model(embeddings)
        sub_predictions.extend(outputs.cpu().numpy())


D. Créer le fichier de soumission

In [ ]:
submission_df = pd.DataFrame({
    "id": src_sub["id"],
    "label": sub_predictions
})

submission_df.head()
submission_df.to_csv("submission.csv", index=False)
print("✅ Fichier de soumission sauvegardé : submission.csv")


Je viens de construire et entraîner un détecteur de texte généré par IA basé sur un GAN et BERT, et tu as généré un fichier de soumission prêt à être utilisé pour une compétition ou évaluation. 💪